# Improving MAPE with CatBoost and Optuna Hyperparameter Tuning

In [15]:

import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_percentage_error
from catboost import CatBoostRegressor
import optuna
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler, PowerTransformer

In [1]:
! pip install optuna catboost


In [16]:
# Load the data
data = pd.read_csv("train.csv")

#
base_features = [col for col in data.columns if col not in [f"BlendProperty{i}" for i in range(1, 11)]]
targets = [f"BlendProperty{i}" for i in range(1, 11)]



In [17]:
# Outlier handling using IQR method
def remove_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    df[column] = df[column].clip(lower_bound, upper_bound)
    return df

for column in data.columns:
    data = remove_outliers(data, column)

In [18]:
# Creating  weighted columns
weighted_data = {}
for i in range(1, 6):
    for j in range(1, 11):
        prop_col = f"Component{i}_Property{j}"
        frac_col = f"Component{i}_fraction"
        weighted_col = f"Weighted_Component{i}_Property{j}"
        weighted_data[weighted_col] = data[prop_col] * data[frac_col]

# Combine
data_with_weighted = pd.concat([data[base_features], pd.DataFrame(weighted_data)], axis=1)
data_with_weighted = data_with_weighted.copy()  # De-fragmented copy

scaler = StandardScaler()
scaled_features = scaler.fit_transform(data_with_weighted)
scaled_df_features = pd.DataFrame(scaled_features, columns=data_with_weighted.columns)


pt = PowerTransformer(method='yeo-johnson')
transformed_targets_df = pd.DataFrame(pt.fit_transform(data[targets]), columns=targets, index=data.index)

In [19]:
# Initial feature selection using Random Forest feature importance
top_features = {}
for target in targets:
    X = scaled_df_features
    y = transformed_targets_df[target]
    rf_initial = RandomForestRegressor(n_estimators=100, random_state=42)
    rf_initial.fit(X, y)
    feature_importance = pd.Series(rf_initial.feature_importances_, index=scaled_df_features.columns).sort_values(ascending=False)
    top_features[target] = feature_importance.head(25).index.tolist()  # Use top 25 features

x_base = scaled_df_features
y = transformed_targets_df

In [ ]:
# Train and evaluate for each target using top 25 features
best_models = {}

for target in targets:
    print(f'\nOptimizing for {target}...')
    

    X_selected = x_base[top_features[target]]
    y_selected = y[target]
    
    def objective(trial):
        params = {
            'iterations': trial.suggest_int('iterations', 500, 3000),
            'depth': trial.suggest_int('depth', 4, 10),
            'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.1)
            # 'random_strength': trial.suggest_int('random_strength', 10, 50),
            # 'bagging_temperature': trial.suggest_loguniform('bagging_temperature', 0.01, 100),
            # 'border_count': trial.suggest_int('border_count', 32, 255),
            # 'silent': True
        }
        
        X_train, X_valid, y_train, y_valid = train_test_split(X_selected, y_selected, test_size=0.2, random_state=42)
        model = CatBoostRegressor(**params)
        model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], early_stopping_rounds=50, verbose=False)
        preds = model.predict(X_valid)
        mape = mean_absolute_percentage_error(y_valid, preds)
        return mape
    
    # Optimize hyperparameters
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=10)
    
    best_params = study.best_trial.params
    print(f'Best MAPE for {target}: {study.best_value:.4f}')
    print(f'Best Parameters for {target}: {best_params}')
    
    # Train final model with best parameters
    final_model = CatBoostRegressor(**best_params)
    final_model.fit(X_selected, y_selected)
    best_models[target] = final_model
    
    # Save the model
    # final_model.save_model(f'catboost_model_{target}.cbm')

[I 2025-07-16 12:35:03,861] A new study created in memory with name: no-name-56c5562c-bf12-40ea-bd80-c3985b07e9c9



Optimizing for BlendProperty1...


C:\Users\swara\AppData\Local\Temp\ipykernel_12672\2119094771.py:15: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.1)
[I 2025-07-16 12:35:16,770] Trial 0 finished with value: 0.43472524689987607 and parameters: {'iterations': 1015, 'depth': 9, 'learning_rate': 0.05659048813346511}. Best is trial 0 with value: 0.43472524689987607.
C:\Users\swara\AppData\Local\Temp\ipykernel_12672\2119094771.py:15: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.1)
[I 2025-07-16 12:35:24,229] Trial 1 finished with value: 0.2915443977839816 and parameters: {'it

Best MAPE for BlendProperty1: 0.2215
Best Parameters for BlendProperty1: {'iterations': 2111, 'depth': 4, 'learning_rate': 0.03285037406834459}
0:	learn: 0.9822278	total: 3.82ms	remaining: 8.05s
1:	learn: 0.9651148	total: 7.45ms	remaining: 7.86s
2:	learn: 0.9482605	total: 12.9ms	remaining: 9.03s
3:	learn: 0.9320438	total: 16.1ms	remaining: 8.51s
4:	learn: 0.9175118	total: 18.3ms	remaining: 7.69s
5:	learn: 0.9028083	total: 20.8ms	remaining: 7.3s
6:	learn: 0.8894118	total: 23.8ms	remaining: 7.14s
7:	learn: 0.8750541	total: 26.2ms	remaining: 6.9s
8:	learn: 0.8612151	total: 28.1ms	remaining: 6.56s
9:	learn: 0.8489185	total: 29.7ms	remaining: 6.24s
10:	learn: 0.8367736	total: 31.3ms	remaining: 5.98s
11:	learn: 0.8258191	total: 33.2ms	remaining: 5.8s
12:	learn: 0.8135849	total: 35.2ms	remaining: 5.68s
13:	learn: 0.8018148	total: 37.7ms	remaining: 5.64s
14:	learn: 0.7903404	total: 39.4ms	remaining: 5.51s
15:	learn: 0.7788173	total: 41.2ms	remaining: 5.39s
16:	learn: 0.7683544	total: 43.1ms	re

[I 2025-07-16 12:37:07,332] A new study created in memory with name: no-name-9d1241b3-db9c-49e8-b78c-68376103e577


...

C:\Users\swara\AppData\Local\Temp\ipykernel_12672\2119094771.py:15: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.1)
[I 2025-07-16 12:43:35,093] Trial 0 finished with value: 0.09544941713165865 and parameters: {'iterations': 2359, 'depth': 6, 'learning_rate': 0.034372941072310356}. Best is trial 0 with value: 0.09544941713165865.
C:\Users\swara\AppData\Local\Temp\ipykernel_12672\2119094771.py:15: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.1)
[I 2025-07-16 12:43:36,723] Trial 1 finished with value: 0.10934200228493308 and parameters: {'

Best MAPE for BlendProperty5: 0.0849
Best Parameters for BlendProperty5: {'iterations': 2893, 'depth': 5, 'learning_rate': 0.04012904871960454}
0:	learn: 0.9652999	total: 3.96ms	remaining: 11.5s
1:	learn: 0.9329746	total: 6.83ms	remaining: 9.88s
2:	learn: 0.9048779	total: 9.45ms	remaining: 9.1s
3:	learn: 0.8772218	total: 15ms	remaining: 10.8s
4:	learn: 0.8475844	total: 17.8ms	remaining: 10.3s
5:	learn: 0.8192846	total: 20.5ms	remaining: 9.86s
6:	learn: 0.7946924	total: 23ms	remaining: 9.48s
7:	learn: 0.7699060	total: 25.7ms	remaining: 9.26s
8:	learn: 0.7452494	total: 29.9ms	remaining: 9.58s
9:	learn: 0.7204715	total: 32.7ms	remaining: 9.44s
10:	learn: 0.6983564	total: 35.4ms	remaining: 9.28s
11:	learn: 0.6758050	total: 37.9ms	remaining: 9.11s
12:	learn: 0.6562473	total: 40.7ms	remaining: 9.03s
13:	learn: 0.6356198	total: 45.4ms	remaining: 9.33s
14:	learn: 0.6144530	total: 47.8ms	remaining: 9.17s
15:	learn: 0.5960957	total: 50ms	remaining: 8.99s
16:	learn: 0.5785928	total: 52.5ms	remain

[I 2025-07-16 12:46:42,475] A new study created in memory with name: no-name-4f0fcd7f-7935-4632-bab9-7863ab0733b1



Optimizing for BlendProperty6...


C:\Users\swara\AppData\Local\Temp\ipykernel_12672\2119094771.py:15: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.1)


In [ ]:
# computing ovrall model  overall MAPE
overall_mape = {}

for target in targets:
    X_selected = x_base[top_features[target]]
    y_selected = y[target]
    
   
    X_train, X_test, y_train, y_test = train_test_split(X_selected, y_selected, test_size=0.2, random_state=42)
    
    
    y_pred = best_models[target].predict(X_test)
    
    # Inverse transform to original scale
    y_test_full = pd.DataFrame(np.column_stack([y_test] + [np.zeros(len(y_test))]*(len(targets)-1)), columns=targets, index=X_test.index)
    y_pred_full = pd.DataFrame(np.column_stack([y_pred] + [np.zeros(len(y_pred))]*(len(targets)-1)), columns=targets, index=X_test.index)
    y_test_inv = pt.inverse_transform(y_test_full)[:, targets.index(target)]
    y_pred_inv = pt.inverse_transform(y_pred_full)[:, targets.index(target)]
    
    # Calculate MAPE on original scale
    mask = y_test_inv != 0
    test_mape = np.mean(np.abs((y_test_inv[mask] - y_pred_inv[mask]) / y_test_inv[mask]) * 100) if mask.any() else 0.0
    overall_mape[target] = test_mape
    
    print(f'Test MAPE for {target} (original scale): {test_mape:.2f}%')

print(f'\nAverage MAPE across all targets: {np.mean(list(overall_mape.values())):.2f}%')